# Model to predict graph embedding from paths

In [2]:
import pickle
from pathlib import Path
with open(Path('F:') / 'data' / 'prov_dp' / 'train_data.pkl', 'rb') as f:
    paths = pickle.load(f)

In [3]:
paths[0]

[('. EPHEMERAL_EPHEMERAL . EPHEMERAL_EPHEMERAL',
  <source.algorithm.wrappers.graph_wrapper.GraphWrapper at 0x2232b7ca710>),
 ('. EPHEMERAL_EPHEMERAL . EPHEMERAL_EPHEMERAL',
  <source.algorithm.wrappers.graph_wrapper.GraphWrapper at 0x2232c10d720>),
 ('. EPHEMERAL_EPHEMERAL . EPHEMERAL_EPHEMERAL',
  <source.algorithm.wrappers.graph_wrapper.GraphWrapper at 0x2232c10d3c0>),
 ('. EPHEMERAL_EPHEMERAL . EPHEMERAL_EPHEMERAL',
  <source.algorithm.wrappers.graph_wrapper.GraphWrapper at 0x2232c10d990>),
 ('. EPHEMERAL_EPHEMERAL . EPHEMERAL_EPHEMERAL',
  <source.algorithm.wrappers.graph_wrapper.GraphWrapper at 0x2234bb38940>),
 ('. EPHEMERAL_EPHEMERAL . EPHEMERAL_EPHEMERAL',
  <source.algorithm.wrappers.graph_wrapper.GraphWrapper at 0x2234bcd7400>),
 ('. EPHEMERAL_EPHEMERAL . EPHEMERAL_EPHEMERAL',
  <source.algorithm.wrappers.graph_wrapper.GraphWrapper at 0x2234c031780>),
 ('. EPHEMERAL_EPHEMERAL . EPHEMERAL_EPHEMERAL',
  <source.algorithm.wrappers.graph_wrapper.GraphWrapper at 0x2234c031930>),
